### Database sanity check notebook
This notebook's purpose is to: 
- Verify Database connectivity 
- Inspect table structure 
- Validate target variable, `no_show  `
- Catch data issues early before EDA and modelling

In [2]:
import sqlite3
import pandas as pd
DB_PATH = "../data/noshow.db"
conn = sqlite3.connect(DB_PATH)

In [3]:
df = pd.read_sql("SELECT * FROM noshow", conn)
df.head()

,booking_id,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
0,94113,0.0,Changi,November,June,25.0,June,27.0,Singapore,Yes,Single,SGD$ 492.98,Website,1,0.0
1,86543,0.0,Orchard,August,November,28.0,November,29.0,Indonesia,Yes,King,SGD$ 1351.22,Website,2,0.0
2,75928,0.0,Changi,March,February,7.0,February,11.0,India,Yes,Single,None,Agent,1,0.0
3,66947,1.0,Orchard,September,October,1.0,October,3.0,China,Yes,Single,SGD$ 666.04,Website,1,0.0
4,106390,0.0,Orchard,March,June,20.0,June,24.0,Australia,Yes,Queen,USD$ 665.37,Website,1,0.0


### Database Sanity Check (EDA PROCESS) - Findings
Dataset contains 119,391 unique bookings across 15 features
- Target variable (`no_show`) has a high baseline rate of ~37%, indicating significant revenue risk
- One row contains missing values across multiple columns, including the target
- Significant missingness observed in `price` (~21%) and `room` (~18%)
- Several numeric fields are incorrectly typed as objects and will require cleaning
- Invalid values detected in `checkout_day` (negative values), indicating data quality issues
- No duplicate booking IDs detected


In [4]:
# Data set contains 119391 rows (Bookings) and 15 columns (Features)
df.shape

(119391, 15)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119391 entries, 0 to 119390
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   booking_id      119391 non-null  int64  
 1   no_show         119390 non-null  float64
 2   branch          119390 non-null  object 
 3   booking_month   119390 non-null  object 
 4   arrival_month   119390 non-null  object 
 5   arrival_day     119390 non-null  float64
 6   checkout_month  119390 non-null  object 
 7   checkout_day    119390 non-null  float64
 8   country         119390 non-null  object 
 9   first_time      119390 non-null  object 
 10  room            97778 non-null   object 
 11  price           94509 non-null   object 
 12  platform        119390 non-null  object 
 13  num_adults      119390 non-null  object 
 14  num_children    119390 non-null  float64
dtypes: float64(4), int64(1), object(10)
memory usage: 13.7+ MB


In [6]:
df.columns

Index(['booking_id', 'no_show', 'branch', 'booking_month', 'arrival_month',
       'arrival_day', 'checkout_month', 'checkout_day', 'country',
       'first_time', 'room', 'price', 'platform', 'num_adults',
       'num_children'],
      dtype='object')

In [7]:
df["no_show"].value_counts(dropna=False)

no_show
0.0    75166
1.0    44224
NaN        1
Name: count, dtype: int64

In [8]:
df["no_show"].mean()
# moderate class imbalance. 1 missing target value. High no-show baseline. 

np.float64(0.37041628277075134)

In [9]:
df.isna().sum().sort_values(ascending=False)
# Significant missing values in 'price' and 'room'. All other columns have 1 missing value. 
# suggests one corrupted row and systematic missingness in 'price' and 'room' columns.

price             24882
room              21613
no_show               1
branch                1
booking_month         1
arrival_month         1
arrival_day           1
checkout_month        1
checkout_day          1
country               1
first_time            1
platform              1
num_adults            1
num_children          1
booking_id            0
dtype: int64

In [10]:
df["booking_id"].nunique(), len(df)

# booking_id is unique identifier. No duplicate rows. Safe to treat each row as one booking. 


(119391, 119391)

In [ ]:
df.describe()
# checkout_day gives negative values. Data quality issue to address later. 

,booking_id,no_show,arrival_day,checkout_day,num_children
count,119391.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,59695.000000,0.370416,15.798241,14.250507,0.871229
std,34465.357332,0.482918,8.780829,11.063697,0.779796
min,0.000000,0.000000,1.000000,-31.000000,0.000000
25%,29847.500000,0.000000,8.000000,7.000000,0.000000
50%,59695.000000,0.000000,16.000000,15.000000,1.000000
75%,89542.500000,1.000000,23.000000,23.000000,1.000000
max,119390.000000,1.000000,31.000000,31.000000,3.000000
